# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

Person(__id__, gender, age)

Drug(__code__, name)

Use(__person_id__, __drug_code__, days_use)

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [15]:
%defaultDatasource jdbc:h2:mem:db

In [16]:
DROP TABLE IF EXISTS Pessoa;
DROP TABLE IF EXISTS Medicamento;
DROP TABLE IF EXISTS Uso;

In [17]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../../../data/nhanes/demographic-person.csv');

In [18]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [19]:
CREATE TABLE Medicamento (
  code VARCHAR(7) NOT NULL,
  name VARCHAR(100) NOT NULL,
  PRIMARY KEY(code)
) AS SELECT
    code,
    name
FROM CSVREAD('../../../data/nhanes/medications-drug.csv');

CREATE TABLE Uso (
  person_id VARCHAR(5) NOT NULL,
  drug_code VARCHAR(7) NOT NULL,
  days_use INTEGER,
  FOREIGN KEY(person_id)
    REFERENCES Pessoa(id)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION,
  FOREIGN KEY(drug_code)
    REFERENCES Medicamento(code)
        ON DELETE NO ACTION
        ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../../../data/nhanes/medications-use.csv');

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [20]:
SELECT id FROM Pessoa P
    WHERE P.age >60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [21]:
SELECT DISTINCT age FROM Pessoa P

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [22]:
SELECT person_id, age, drug_code, days_use FROM Pessoa P, Uso
    WHERE Uso.person_id = P.id

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [23]:
SELECT person_id, age, name, days_use FROM Pessoa P, Uso, Medicamento M
    WHERE Uso.person_id = P.id AND M.code = Uso.drug_code 

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [24]:
SELECT name FROM Medicamento M, Uso
    WHERE Uso.drug_code = M.code
    GROUP BY name

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [25]:
SELECT name FROM Medicamento M, Uso
    WHERE Uso.drug_code = M.code AND Uso.days_use > 2000
    GROUP BY name